# Building graph

In [1]:
import json

import networkx as nx

In [2]:
graph = nx.MultiDiGraph()

## Nodes

In [3]:
for c in ['ent', 'per', 'pub', 'emp', 'exp']:
    with open('data/' + c + '-posts.json') as f:
        js = json.load(f)
        for e in js[1:]:
            ID = str(e['ID'])
            typ = e['post_type']
            if typ == 'entity':
                name = e['post_title']
                keyword = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_keyword'])
                typology = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_typology'])
                ownership = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_ownership'])
                nationality = ''
                if '_cp__ent_town' in e['meta']:
                    nationality  = e['meta']['_cp__ent_town'][0].split('; ')[-1]
                data = {
                    'id': ID,
                    'type': typ,
                    'name': name,
                    'keyword': keyword,
                    'typology': typology,
                    'ownership': ownership,
                    'nationality': nationality,
                }
            elif typ == 'person':
                name = e['post_title']
                activity = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_activity'])
                chronology = ''
                nationality = ''
                if '_cp__ent_town' in e['meta']:
                    nationality  = e['meta']['_cp__peo_country'][0].split('; ')[-1]
                data = {
                    'id': ID,
                    'type': typ,
                    'name': name,
                    'activity': activity,
                    'chronology': chronology,
                    'nationality': nationality,
                }
            elif typ == 'book':
                title = e['post_title']
                date = ''
                if '_cp__boo_publishing_date' in e['meta']:
                    date = e['meta']['_cp__boo_publishing_date'][0]
                publisher = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_publisher'])
                data = {
                    'id': ID,
                    'type': typ,
                    'title': title,
                    'date': date,
                    'publisher': publisher,
                }
            elif typ == 'company':
                name = e['post_title']
                activity = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_isic4_category'])
                nationality = e['meta']['_cp__com_company_headquarter_place'][0].split('; ')[-1]
                data = {
                    'id': ID,
                    'type': typ,
                    'name': name,
                    'activity': activity,
                    'nationality': nationality,
                }
            elif typ == 'exhibition':
                title = e['post_title']
                movement = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_movement'])
                period = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_period'])
                topic = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_topic'])
                artwork_type = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_artwork_type'])
                typology = '; '.join([t['name'] for t in e['terms'] if t['taxonomy'] == 'tax_exhibition_type'])
                nationality = ''
                if '_cp__exh_exhibition_town' in e['meta']:
                    nationality = e['meta']['_cp__exh_exhibition_town'][0].split('; ')[-1]
                data = {
                    'id': ID,
                    'type': typ,
                    'title': title,
                    'movement': movement,
                    'period': period,
                    'topic': topic,
                    'artwork_type': artwork_type,
                    'typology': typology,
                    'nationality': nationality,
                }
            graph.add_node(ID, data)

## Relationships

In [4]:
highest_id = str(max(int(ide) for ide in graph))

In [5]:
def increase_id():
    global highest_id
    highest_id = str(int(highest_id) + 1)

In [6]:
def get_id_and_name(s):
    if ': ' in s:
        ide = s.split(': ')[0]
        name = s.split(': ')[1]
    else:
        increase_id()
        ide = highest_id
        name = s
    return (ide, name)

In [7]:
for c in ['ent', 'per', 'pub', 'emp', 'exp']:
    with open('data/' + c + '-posts.json') as f:
        js = json.load(f)
        for e in js[1:]:
            ID = str(e['ID'])
            typ = e['post_type']
            if typ == 'entity':
                pass
            elif typ == 'person':
                if '_cp__peo_entity_relation' in e['meta']:
                    for entity in e['meta']['_cp__peo_entity_relation']:
                        ide, name  = get_id_and_name(entity)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'entity',
                                'name': name,
                                'keyword': '',
                                'typology': '',
                                'ownership': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ID, ide, label='_cp__peo_entity_relation')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ID, ide, label='_cp__peo_entity_relation')
            elif typ == 'book':
                if '_cp__boo_paper_author' in e['meta']:
                    for author in e['meta']['_cp__boo_paper_author']:
                        ide, name  = get_id_and_name(author)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'person',
                                'name': name,
                                'activity': activity,
                                'chronology': chronology,
                                'nationality': nationality,
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__boo_paper_author')
                        elif graph.node[ide]['type'] == 'person':
                            graph.add_edge(ide, ID, label='_cp__boo_paper_author')
                if '_cp__boo_sponsorship' in e['meta']:
                    for entity in e['meta']['_cp__boo_sponsorship']:
                        ide, name  = get_id_and_name(entity)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'entity',
                                'name': name,
                                'keyword': '',
                                'typology': '',
                                'ownership': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__boo_sponsorship')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ide, ID, label='_cp__boo_sponsorship')
            elif typ == 'company':
                pass
            elif typ == 'exhibition':
                if '_cp__exh_info_source' in e['meta']:
                    for entity in e['meta']['_cp__exh_info_source']:
                        ide, name  = get_id_and_name(entity)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'entity',
                                'name': name,
                                'keyword': '',
                                'typology': '',
                                'ownership': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_info_source')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ide, ID, label='_cp__exh_info_source')
                if '_cp__exh_supporter_entity' in e['meta']:
                    for entity in e['meta']['_cp__exh_supporter_entity']:
                        ide, name  = get_id_and_name(entity)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'entity',
                                'name': name,
                                'keyword': '',
                                'typology': '',
                                'ownership': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_supporter_entity')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ide, ID, label='_cp__exh_supporter_entity')
                if '_cp__exh_funding_entity' in e['meta']:
                    for entity in e['meta']['_cp__exh_funding_entity']:
                        ide, name  = get_id_and_name(entity)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'entity',
                                'name': name,
                                'keyword': '',
                                'typology': '',
                                'ownership': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_funding_entity')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ide, ID, label='_cp__exh_funding_entity')
                if '_cp__exh_artwork_author' in e['meta']:
                    for person in e['meta']['_cp__exh_artwork_author']:
                        ide, name  = get_id_and_name(person)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'person',
                                'name': name,
                                'activity': '',
                                'chronology': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_artwork_author')
                        elif graph.node[ide]['type'] == 'person':
                            graph.add_edge(ide, ID, label='_cp__exh_artwork_author')
                if '_cp__exh_curator' in e['meta']:
                    for person in e['meta']['_cp__exh_curator']:
                        ide, name  = get_id_and_name(person)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'person',
                                'name': name,
                                'activity': '',
                                'chronology': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_curator')
                        elif graph.node[ide]['type'] == 'person':
                            graph.add_edge(ide, ID, label='_cp__exh_curator')
                if '_cp__exh_art_collector' in e['meta']:
                    for person in e['meta']['_cp__exh_art_collector']:
                        ide, name  = get_id_and_name(person)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'person',
                                'name': name,
                                'activity': '',
                                'chronology': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_art_collector')
                        elif graph.node[ide]['type'] == 'person':
                            graph.add_edge(ide, ID, label='_cp__exh_art_collector')
                        elif graph.node[ide]['type'] == 'entity':
                            graph.add_edge(ide, ID, label='entity_art_collector') #
                if '_cp__exh_museography' in e['meta']:
                    for company in e['meta']['_cp__exh_museography']:
                        ide, name  = get_id_and_name(company)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'company',
                                'name': name,
                                'activity': '',
                                'nationality': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ide, ID, label='_cp__exh_museography')
                        elif graph.node[ide]['type'] == 'company':
                            graph.add_edge(ide, ID, label='_cp__exh_museography')
                if '_cp__exh_catalog' in e['meta']:
                    for book in e['meta']['_cp__exh_catalog']:
                        ide, name  = get_id_and_name(book)
                        if ide not in graph:
                            data = {
                                'id': ide,
                                'type': 'book',
                                'title': name,
                                'publisher': '',
                                'date': '',
                            }
                            graph.add_node(ide, data)
                            graph.add_edge(ID, ide, label='_cp__exh_catalog')
                        elif graph.node[ide]['type'] == 'book':
                            graph.add_edge(ID, ide, label='_cp__exh_catalog')

## Reduction

In [8]:
from collections import Counter
from collections import defaultdict

sizes = defaultdict(int)
for ide, data in graph.nodes_iter(data=True):
    sizes[data['type']] += 1

total = defaultdict(list)
for ide, data in graph.nodes_iter(data=True):
    deg = graph.degree(ide)
    res = [ide] * deg
    total[data['type']].extend(res)

freqs = {}
for k in total:
    length = len(set(total[k]))
    freqs[k] = Counter(total[k]).most_common(max(length // 3, 1))

new_nodes = set()
for k in freqs:
    for ide, fr in freqs[k]:
        new_nodes.add(ide)

for n in graph.nodes():
    if n not in new_nodes:
        graph.remove_node(n)

In [9]:
len(graph)

7179

# Exporting to Sylva

## Mappings

In [10]:
# Classes
classes = {
    'entity': 'Entidad',
    'person': 'Persona',
    'book': 'Publicación',
    'company': 'Empresa',
    'exhibition': 'Exposición',
}

# Attributes
attributes = {
    'entity': {
        'name': 'nombre',
        'keyword': 'palabra clave',
        'typology': 'tipología',
        'ownership': 'titularidad',
        'nationality': 'país',
    },
    'person': {
        'name': 'nombre',
        'activity': 'actividad',
        'chronology': 'cronología',
        'nationality': 'nacionalidad',
    },
    'book': {
        'title': 'título',
        'publisher': 'editorial',
        'date': 'fecha',
    },
    'company': {
        'name': 'nombre',
        'activity': 'actividad',
        'nationality': 'nacionalidad',
    },
    'exhibition': {
        'title': 'título',
        'movement': 'movimiento',
        'period': 'período',
        'topic': 'tema',
        'artwork_type': 'tipo de artefactos',
        'typology': 'tipología',
        'nationality': 'país',
    },
}

# Relationships
relationships = {
    '_cp__peo_entity_relation': 'pertenece a',
    '_cp__boo_paper_author': 'autor de',
    '_cp__boo_sponsorship': 'financia publicación',
    '_cp__exh_info_source': 'fuente de información',
    '_cp__exh_supporter_entity': 'organiza',
    '_cp__exh_funding_entity': 'financia exposición',
    '_cp__exh_artwork_author': 'participa en',
    '_cp__exh_curator': 'comisaria',
    '_cp__exh_art_collector': 'presta colecciones',
    '_cp__exh_museography': 'produce (museografía)',
    '_cp__exh_catalog': 'se inscribe en',
    'entity_art_collector': 'presta colecciones expo', #
}

## File system

In [11]:
import csv
import os
import shutil
from collections import defaultdict

In [12]:
shutil.rmtree('sylva/nodes', ignore_errors=True)
shutil.rmtree('sylva/relationships', ignore_errors=True)
os.makedirs('sylva/nodes')
os.makedirs('sylva/relationships')

In [13]:
for k in classes:
    with open(os.path.join('sylva/nodes', classes[k] + '.csv'), 'w') as f:
        writer = csv.writer(
            f,
            delimiter=',',
            quotechar='"',
            quoting=csv.QUOTE_ALL
        )
        attr_items = attributes[k].items()
        header = ['id', 'type'] + [x[1] for x in attr_items]
        writer.writerow(header)
        attrs = ['id', 'type'] + [x[0] for x in attr_items]
        for ide, data in graph.nodes_iter(data=True):
            if data['type'] == k:
                row = [data[a] if a != 'type' else classes[data[a]] for a in attrs]
                writer.writerow(row)
for k in relationships:
    with open(os.path.join('sylva/relationships', relationships[k] + '.csv'), 'w') as f:
        writer = csv.writer(
            f,
            delimiter=',',
            quotechar='"',
            quoting=csv.QUOTE_ALL
        )
        header = ['source id', 'target id', 'label']
        writer.writerow(header)
        attrs = ['id', 'type'] + [x[0] for x in attr_items]
        for id1, id2, data in graph.edges_iter(data=True):
            if data['label'] == k:
                row = [id1, id2, relationships[k]]
                writer.writerow(row)

In [13]:
with open(os.path.join('sylva/relationships/autor de.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'person':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'book':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'autor de':
            print('3', graph[row[0]][row[1]])
            break

In [14]:
with open(os.path.join('sylva/relationships/comisaria.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'person':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'comisaria':
            print('3', graph[row[0]][row[1]])
            break

3 {0: {'label': '_cp__exh_artwork_author'}, 1: {'label': '_cp__exh_curator'}}


In [15]:
with open(os.path.join('sylva/relationships/financia exposición.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'entity':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'financia exposición':
            print('3', graph[row[0]][row[1]])
            break

3 {0: {'label': '_cp__exh_supporter_entity'}, 1: {'label': '_cp__exh_funding_entity'}}


In [16]:
with open(os.path.join('sylva/relationships/financia publicación.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'entity':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'book':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'financia publicación':
            print('3', graph[row[0]][row[1]])
            break

In [17]:
with open(os.path.join('sylva/relationships/fuente de información.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'entity':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'fuente de información':
            print('3', graph[row[0]][row[1]])
            break

In [18]:
with open(os.path.join('sylva/relationships/organiza.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'entity':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'organiza':
            print('3', graph[row[0]][row[1]])
            break

3 {0: {'label': '_cp__exh_info_source'}, 1: {'label': '_cp__exh_supporter_entity'}}


In [19]:
with open(os.path.join('sylva/relationships/participa en.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'person':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'participa en':
            print('3', graph[row[0]][row[1]])
            break

In [20]:
with open(os.path.join('sylva/relationships/pertenece a.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'person':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'entity':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'pertenece a':
            print('3', graph[row[0]][row[1]])
            break

In [21]:
with open(os.path.join('sylva/relationships/presta colecciones.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'person':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'presta colecciones':
            print('3', graph[row[0]][row[1]])
            break

In [22]:
with open(os.path.join('sylva/relationships/presta colecciones expo.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'entity':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'presta colecciones expo':
            print('3', graph[row[0]][row[1]])
            break

3 {0: {'label': '_cp__exh_info_source'}, 1: {'label': '_cp__exh_supporter_entity'}, 2: {'label': 'entity_art_collector'}}


In [23]:
with open(os.path.join('sylva/relationships/produce (museografía).csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'company':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'exhibition':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'produce (museografía)':
            print('3', graph[row[0]][row[1]])
            break

In [24]:
with open(os.path.join('sylva/relationships/se inscribe en.csv')) as f:
    reader = csv.reader(
        f,
        delimiter=',',
        quotechar='"'
    )
    next(reader, None)
    for row in reader:
        if graph.node[row[0]]['type'] != 'exhibition':
            print('1', graph.node[row[0]])
            break
        if graph.node[row[1]]['type'] != 'book':
            print('2', graph.node[row[1]])
            break
        if relationships[list(list(graph[row[0]][row[1]].values())[0].values())[0]] != 'se inscribe en':
            print('3', graph[row[0]][row[1]])
            break

In [25]:
for e in os.listdir('sylva/nodes/'):
    with open(os.path.join('sylva/nodes/', e)) as f:
        reader = csv.reader(
            f,
            delimiter=',',
            quotechar='"'
        )
        next(reader, None)
        for row in reader:
            if classes[graph.node[row[0]]['type']] != e[:-4]:
                print('ERROR')
                break

In [26]:
# from collections import Counter
# from collections import defaultdict

# sizes = defaultdict(int)
# for ide, data in graph.nodes_iter(data=True):
#     sizes[data['type']] += 1

# total = defaultdict(list)
# for ide, data in graph.nodes_iter(data=True):
#     deg = graph.degree(ide)
#     res = [ide] * deg
#     total[data['type']].extend(res)

# freqs = {}
# for k in total:
#     length = len(set(total[k]))
#     freqs[k] = Counter(total[k]).most_common(max(length // 10, 1))

# new_nodes = set()
# for k in freqs:
#     for ide, fr in freqs[k]:
#         new_nodes.add(ide)

# for n in graph.nodes():
#     if n not in new_nodes:
#         graph.remove_node(n)